In [3]:
# Import the requests library.
import requests

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Import the API key.
from config import weather_api_key

# from citipy import citipy